# VPC Components  
>A *VPC (Virtual Private Cloud)* is a secure, isolated cloud environment hosted within a public cloud. A VPC allows you to group together cloud resources and control the networking environment around them. 

There are various components needed to build and operate a VPC, which will be covered in this notebook. These components will be discussed generally first of all, and then in the context of Amazon VPCs more specifically.

These are the key features & benefits of a VPC:

- Comprehensive control over the virtual networking environment of groups of resources

- Allows for the division of a large network into smaller parts
- Allows for granular control of the flow of traffic in and out of specific CIDR blocks, including automatically determining where that traffic should go depending on it's volume
- Sophisticated network monitoring and logging capabilities 
- Easy to implement a hybrid cloud deployment by connecting the VPC to an on-premises infrastructure 

### Use Cases of a VPC
- **Hosting a public facing website:** When hosting a website, you may require multiple compute instances along with a database. Using a VPC allows you to add layers of security and traffic monitoring so you can control the inbound and outbound traffic to your site, avoiding attacks.

- **Connecting branch offices with a cloud system:** There are various different options for connecting remote, on-premises systems to a VPC. This would allow you to benefit from the high availability and reliability of cloud resources, while being fully connected to your on-premises system.

- **Development and testing environments:** Creating isolated VPCs for different environments ensures that testing does not affect the production environment and allows for secure and controlled development processes.

The following diagram illustrates at a high level the components of a VPC we will be discussing in this notebook. Refer back to this as you progress through the different concepts.

<p align="center"> <img src="images/VPC-Components.jpeg" height="588" width="880"/> </p>

## Subnets
>*Subnets* are subdivisions of the IP address range defined for the VPC. They allow you to split the VPC into smaller sub-networks, or subnets. This is useful because it means you can control the networking around cloud resources at a more granular level. Subnets can be configured to be either public or private: accessible from the internet or not, depending on the use case of the resources inside each subnet.

When configuring a subnet in an Amazon VPC, you can allow AWS to auto-assign public IP addresses to instances launched in that subnet. This does not mean those instances are then publicly accessible from the internet; the public IP address assigned to the instances are picked from a pool of protected, public IP addresses owned by Amazon.

Before discussing public and private subnets in more depth, it's important to understand what an Internet Gateway is.

>An *Internet Gateway (IGW)* is an AWS service designed to facilitate bidirectional communication between resources within the VPC and the internet. It allows inbound traffic from the internet to reach resources hosted in the VPC and enables outbound traffic from resources within the VPC to reach the internet. An Internet Gateway is also a horizontally scalable and highly available service.

### Public vs Private Subnets
>A *public subnet* is defined by AWS as one which has a direct route to an Internet Gateway, and can therefore access the internet and be accessed from the internet. A *private subnet* is one which does not have a direct route to an Internet Gateway.


For example, a private subnet would be used for resources such as database servers which need high levels of protection, and are only accessed by other resources in your VPC so no public internet connection is required. Public subnets are used for resources which need to be accessed from the internet, such as public-facing web servers.

## NAT Services
The primary benefit of private subnets is that they enhance security by preventing direct access from the internet, since they are not connected to an Internet Gateway. However, without an Internet Gateway, instances in a private subnet can not send outbound traffic either, because they don't have a publicly routable IP address to set as the reply field. To solve this problem, we need to perform NAT for any instances in private subnets.

>As a reminder, *NAT (Network Access Translation)* is the process of assigning public IP addresses to devices and instances so they can be accessed outside a private network such as a private subnet or LAN. Without NAT, resources in a private subnet cannot communicate with anything outside of that VPC. 

Traditionally, you could provision a special type of EC2 instance in a public subnet which would perform NAT and allow instances in a private subnet to send outbound traffic. This method is known as a *NAT Instance*. Although this is still possible, it less common today because it is difficult to scale and is unreliable: if that EC2 instance fails then your private subnet is cut off. As a solution to this, Amazon created NAT Gateways.

### NAT Gateways
>A *NAT Gateway* is an AWS service which performs NAT on instances in private subnets so that they have public IP addresses, and can therefore connect to the internet. This means that multiple instances can share the same public IP address. Each NAT Gateway is created in a specific Availability Zone, with redundancy within that Availability Zone. They can also scale automatically and can handle large volumes of traffic. 

When you create a NAT Gateway, you specify one of the following connectivity types:

- **Public:** Choosing this option allows instances in private subnets to connect to the internet, but cannot receive unsolicited inbound connections. This method requires the NAT Gateway to be provisioned in a public subnet, because it works in conjunction with an Internet Gateway to allow outbound traffic from the private subnet to the internet. 

- **Private:** Private NAT Gateways only allow traffic from instances to other VPCs on your AWS account or a connected on-premises network. Any traffic sent from a private NAT Gateway through an Internet Gateway will be automatically dropped.

<p align="center"> <img src="images/NAT-Gateways.jpeg" height="630" width="539"/> </p>

#### Elastic IP Addresses
>An *elastic IP address* in AWS is a public IPv4 address which you can associate with an instance in your VPC, and if that instance fails the elastic IP address can quickly and automatically remap itself to another instance within the same VPC. This is useful because it means you can ensure very little downtime for an application or server, should an instance fail.

When you create a public NAT gateway you must associate an elastic IP address with the NAT gateway at creation. AWS does this automatically for you by default, but you can create and assign your own. You cannot associate an elastic IP address with a private NAT gateway.

## Route Tables
>A *route table* contains a set of rules, called routes, that determine where network traffic from your subnet is directed. When you create a VPC with AWS you get a route table by default, called the *main route table*, which you can add routes to, or you can create your own *custom route table*.

The following are the key concepts you need to know about route tables:

- **Destination:** This is the first column in a route table, it is the range of IP addresses where you want the traffic to go, defined using CIDR notation.

- **Target:** The second column in a route table, this is the connection through which the traffic should go, such as a NAT Gateway, Internet Gateway or instance ID (for traffic directed to specific instances in a VPC).

- **Route priority:** Routes are evaluated in priority order, with more specific routes taking precedence over less specific ones

- **Route table association:** The association between a route table and a subnet or internet gateway. 

Consider the following use cases and route table configurations:

**Example 1:** For a web server to be able to serve content to users on the internet, a route table would be requireed to allow traffic from your subnet to anywhere on the public internet. The following route would achieve this: 

Destination | Target
------------|--------
`0.0.0.0/0` |`internetgateway-id`


**Example 2:** In this next example, a database server in a private subnet needs access to the internet for software updates, but should be directly accessible from the internet. The following route table directs non-local traffic to a NAT Gateway in a public subnet, allowing outbound internet access while keeping inbound traffic restricted.

Destination | Target
------------|--------
`10.0.0.0/16` |`local` (default route for internal VPC communication)
`0.0.0.0/0` | `nat-123456`
    
**Example 3:** In this route table, traffic destined for a subnet with the `172.9.31.0/20` CIDR block is routed to a specific network interface. Traffic to all other subnets in the VPC uses the local route.

Destination | Target
------------|--------
`172.9.31.0/16` | Local
`172.9.31.0/20` | `eni-id`

## Hands-On: Creating a Custom VPC With Public and Private Subnets
AWS automatically provides a *default VPC* in each AWS region. The default VPC simplifies the process of getting started with AWS resources by providing pre-configured settings and infrastructure. It includes a pre-defined CIDR block and a public subnets. However, you also have the option to create a *custom VPC*. A custom VPC allows for more control and customization, giving you the flexibility to define your own CIDR block and subnets.

When creating a VPC, you define an IP address range using CIDR notation. AWS recommends that you use a private IPv4 address range for setting up your VPC, although it is possible to define a publicly routable CIDR block instead. Doing this would require you to own or have access to this public range of IP addresses, however. Also, for the remainder of this notebook we will discuss VPCs as if they have been defined with a private IPv4 address range.
<br></br>

### Creating the Custom VPC

1. After logging in to the AWS console, navigate to VPC using the search bar and select *Create VPC*

2. On the next screen, select *VPC only*. We are going to manually create the other resources in a later Hands-On exercise.
3. Enter a name for your VPC
4. IPv4 CIDR block: Select *IPv4 CIDR manual input*
5. Define your CIDR block. For the purposes of this Hands-On we will use `10.0.0.0/24`, which contains 256 IP addresses.
6. IPv6 CIDR block: Select *No IPv6 CIDR block*
6. Tenancy: Select *Default*. This attribute determines if instances that are launched into the VPC will run on hardware that's shared with other AWS accounts, or on dedicated for your use only.
8. Press *Create VPC*

<p align="center"> <img src="images/1-vpc-settings.png" height="587" width="508"/> </p>

Once you have created your VPC you will be redirected back to the *Your VPCs* page. You should be able to see two VPCs in the list: the first one is the default VPC that comes with your AWS account, the second is the one we just created, with the IPv4 CIDR block of `10.0.0.0/24`.

### Creating the Subnets
In this step we are going to create two subnets, which will be configured as public and private later.

1. From the main VPC dashboard, on the sidebar, click *Subnets*. There will be three subnets listed here already. These are for the default VPC, one subnet for each of the availability zones in your region.

2. Press *Create Subnet*
3. Select the VPC we created a moment ago
4. Enter a name. This step is actually optional, but doing so allows you to more easily differentiate between subnets. Call this first subnet `my-test-public-subnet-1`, for example
5. Choose an Availability Zone
6. Enter `10.0.0.0/28` for the *IPv4 subnet CIDR block*. This subnet has 16 IP addresses.
7. Leave everything else as it is

8. Complete this step once again to create a second subnet, but calling it something like `my-test-private-subnet-1` instead. Make sure to select a different CIDR block so it doesn't overlap with the public subnet.

<p align="center"> <img src="images/2-subnet-settings.png" height="587" width="508"/> </p>

### Configure the Public subnet
In this step we are going to allow the instances in this subnet to be auto-assigned a public IP address owned by Amazon.

1. Select your public subnet

2. Press *Actions* then *Edit subnet settings*
3. Select the box for *Enable auto-assign public IPv4 address*

### Create the Internet Gateway
In this step we are going to create an Internet Gateway which will allow the public subnet to communicate with the internet.

1. From the main VPC dashboard, on the sidebar, press *Internet Gateways*

2. There will be one there already, this is the default IGW. Click *Create internet gateway*.
3. Enter a name. Press *Create Internet Gateway*.
4. On the next screen, press *Actions*, then *Attach to a VPC*
5. Select the VPC you created

### Create the NAT Gateway
1. From the main VPC dashboard, on the sidebar, press *NAT Gateways*, then *Create NAT Gateway*

2. Enter a name for your NAT Gateway
3. Select the public subnet you created
4. For connectivity type, select *Public*
5. Press *Allocate Elastic IP*
6. Press *Create NAT gateway*

<p align="center"> <img src="images/5-create-nat-gateway.png" height="536" width="505"/> </p>

### Create Route Tables
The last step of this process is to create custom route tables, and then associate them with the relevant subnet. The private subnet will be routed to the NAT Gateway you just created, and the public subnet will be routed to the Internet Gateway.

1. From the main VPC dashboard, on the sidebar, press *Route Tables*

2. Select *Create route table*
3. Enter a name corresponding to one of the subnets (`public-route-table`, for example). Select your VPC, then press *Create route table*.
4. From the next screen, press *Subnet associations*, then the *Edit subnet associations* button underneath *Explicit subnet associations* (see below screenshot)

<p align="center"> <img src="images/6-edit-subnet-associations.png" height="533" width="1083"/> </p>

5. On the next screen, select the corresponding subnet, then press *Save assocations*

6. Next, in the *Routes* pane of the main route table page, press *Edit routes*
7. Press *Add route*, then enter `0.0.0.0/0` under *Destination* and select the Internet Gateway you created for *Target*. This will allow any traffic out of the subnet to go through the Internet Gateway and be able to commmunicate with the internet. Press *Save changes*.
8. Repeat this whole step for the private subnet and route table, but make the NAT Gateway you have created the target in the route table instead of the Internet Gateway.

## Key Takeaways
- A VPC is a secure, isolated cloud environment within a public cloud. It allows for comprehensive control of the networking environment surrounding groups of resources, by defining a range of IP addresses which you have complete control over.

- Subnets are subdivisions of that IP address range, they can be made public or private
- An Internet Gateway is a service which acts as a link between a VPC and the internet. It is required in order for instances in subnets to connect to the internet, other VPCs or an on-premises network.
- A NAT Gateway is a managed service which allows a private subnet to send outbound traffic to the internet, other VPCs or a connected on-premises network. They can be configured with public or private connectivity: a public NAT Gateway creates a connection with the internet, a private NAT Gateway only allows communication with other VPCs on your account and a connected on-premises network.
- An elastic IP address is a public IPv4 address owned by Amazon which upon failure of an instance it's associated with, can automatically remap itself to another instance to avoid downtime
- A route table is a set of rules which determine where network traffic from your subnet should be directed
- An elastic load balancer is a service which provides a single point of contact in your VPC to then distribute traffic across different targets such as EC2 instances. There are three types of load balancer you need to know about: network, application and gateway.